# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 21 2022 12:38PM,240089,102,PBR-061722-01,"Senseonics, Incorporated",Released
1,Jun 21 2022 12:33PM,240088,10,SONSB0001824,"Nextsource Biotechnology, LLC",Released
2,Jun 21 2022 12:20PM,240086,20,8037114-FL,"ACI Healthcare USA, Inc.",Released
3,Jun 21 2022 12:20PM,240086,20,ACI-8114351-FLS,"ACI Healthcare USA, Inc.",Released
4,Jun 21 2022 12:20PM,240086,20,8138302-FLS,"ACI Healthcare USA, Inc.",Released
5,Jun 21 2022 12:20PM,240086,20,8138303-FLS,"ACI Healthcare USA, Inc.",Released
6,Jun 21 2022 12:20PM,240086,20,8138400-FLS,"ACI Healthcare USA, Inc.",Released
7,Jun 21 2022 12:20PM,240086,20,8138414-FLS,"ACI Healthcare USA, Inc.",Released
8,Jun 21 2022 12:20PM,240086,20,8139066-FLS,"ACI Healthcare USA, Inc.",Released
9,Jun 21 2022 12:20PM,240086,20,8139093-FLS,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
49,240084,Released,29
50,240085,Released,12
51,240086,Released,17
52,240088,Released,1
53,240089,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240084,NaN,NaN,29.0
240085,NaN,NaN,12.0
240086,NaN,NaN,17.0
240088,NaN,NaN,1.0
240089,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239982,29.0,10.0,1.0
239992,0.0,0.0,1.0
240002,1.0,11.0,2.0
240003,0.0,0.0,1.0
240004,0.0,1.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239982,29,10,1
239992,0,0,1
240002,1,11,2
240003,0,0,1
240004,0,1,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239982,29,10,1
1,239992,0,0,1
2,240002,1,11,2
3,240003,0,0,1
4,240004,0,1,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239982,29,10,1
1,239992,,,1
2,240002,1,11,2
3,240003,,,1
4,240004,,1,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 21 2022 12:38PM,240089,102,"Senseonics, Incorporated"
1,Jun 21 2022 12:33PM,240088,10,"Nextsource Biotechnology, LLC"
2,Jun 21 2022 12:20PM,240086,20,"ACI Healthcare USA, Inc."
19,Jun 21 2022 12:19PM,240085,20,"ACI Healthcare USA, Inc."
31,Jun 21 2022 11:58AM,240084,10,Eywa Pharma Inc.
60,Jun 21 2022 11:44AM,240081,20,"ACI Healthcare USA, Inc."
97,Jun 21 2022 11:41AM,240080,10,"Methapharm, Inc."
98,Jun 21 2022 11:29AM,240079,10,ISDIN Corporation
110,Jun 21 2022 11:29AM,240077,20,Alumier Labs Inc.
149,Jun 21 2022 11:25AM,240078,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 21 2022 12:38PM,240089,102,"Senseonics, Incorporated",,,1
1,Jun 21 2022 12:33PM,240088,10,"Nextsource Biotechnology, LLC",,,1
2,Jun 21 2022 12:20PM,240086,20,"ACI Healthcare USA, Inc.",,,17
3,Jun 21 2022 12:19PM,240085,20,"ACI Healthcare USA, Inc.",,,12
4,Jun 21 2022 11:58AM,240084,10,Eywa Pharma Inc.,,,29
5,Jun 21 2022 11:44AM,240081,20,"ACI Healthcare USA, Inc.",,,37
6,Jun 21 2022 11:41AM,240080,10,"Methapharm, Inc.",,,1
7,Jun 21 2022 11:29AM,240079,10,ISDIN Corporation,,,12
8,Jun 21 2022 11:29AM,240077,20,Alumier Labs Inc.,,25,14
9,Jun 21 2022 11:25AM,240078,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 12:38PM,240089,102,"Senseonics, Incorporated",1,,
1,Jun 21 2022 12:33PM,240088,10,"Nextsource Biotechnology, LLC",1,,
2,Jun 21 2022 12:20PM,240086,20,"ACI Healthcare USA, Inc.",17,,
3,Jun 21 2022 12:19PM,240085,20,"ACI Healthcare USA, Inc.",12,,
4,Jun 21 2022 11:58AM,240084,10,Eywa Pharma Inc.,29,,
5,Jun 21 2022 11:44AM,240081,20,"ACI Healthcare USA, Inc.",37,,
6,Jun 21 2022 11:41AM,240080,10,"Methapharm, Inc.",1,,
7,Jun 21 2022 11:29AM,240079,10,ISDIN Corporation,12,,
8,Jun 21 2022 11:29AM,240077,20,Alumier Labs Inc.,14,25,
9,Jun 21 2022 11:25AM,240078,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 12:38PM,240089,102,"Senseonics, Incorporated",1,,
1,Jun 21 2022 12:33PM,240088,10,"Nextsource Biotechnology, LLC",1,,
2,Jun 21 2022 12:20PM,240086,20,"ACI Healthcare USA, Inc.",17,,
3,Jun 21 2022 12:19PM,240085,20,"ACI Healthcare USA, Inc.",12,,
4,Jun 21 2022 11:58AM,240084,10,Eywa Pharma Inc.,29,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 12:38PM,240089,102,"Senseonics, Incorporated",1.0,NaN,NaN
1,Jun 21 2022 12:33PM,240088,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
2,Jun 21 2022 12:20PM,240086,20,"ACI Healthcare USA, Inc.",17.0,NaN,NaN
3,Jun 21 2022 12:19PM,240085,20,"ACI Healthcare USA, Inc.",12.0,NaN,NaN
4,Jun 21 2022 11:58AM,240084,10,Eywa Pharma Inc.,29.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 12:38PM,240089,102,"Senseonics, Incorporated",1.0,0.0,0.0
1,Jun 21 2022 12:33PM,240088,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
2,Jun 21 2022 12:20PM,240086,20,"ACI Healthcare USA, Inc.",17.0,0.0,0.0
3,Jun 21 2022 12:19PM,240085,20,"ACI Healthcare USA, Inc.",12.0,0.0,0.0
4,Jun 21 2022 11:58AM,240084,10,Eywa Pharma Inc.,29.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4800984,222.0,42.0,29.0
102,240089,1.0,0.0,0.0
15,1440359,51.0,45.0,0.0
16,240070,2.0,0.0,0.0
18,240025,0.0,1.0,0.0
19,960086,6.0,12.0,3.0
20,1200336,81.0,25.0,0.0
21,239992,1.0,0.0,0.0
22,240003,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4800984,222.0,42.0,29.0
1,102,240089,1.0,0.0,0.0
2,15,1440359,51.0,45.0,0.0
3,16,240070,2.0,0.0,0.0
4,18,240025,0.0,1.0,0.0
5,19,960086,6.0,12.0,3.0
6,20,1200336,81.0,25.0,0.0
7,21,239992,1.0,0.0,0.0
8,22,240003,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,222.0,42.0,29.0
1,102,1.0,0.0,0.0
2,15,51.0,45.0,0.0
3,16,2.0,0.0,0.0
4,18,0.0,1.0,0.0
5,19,6.0,12.0,3.0
6,20,81.0,25.0,0.0
7,21,1.0,0.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,222.0
1,102,Released,1.0
2,15,Released,51.0
3,16,Released,2.0
4,18,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,29.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
Executing,42.0,0.0,45.0,0.0,1.0,12.0,25.0,0.0,0.0
Released,222.0,1.0,51.0,2.0,0.0,6.0,81.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,18,19,20,21,22
0,Completed,29.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
1,Executing,42.0,0.0,45.0,0.0,1.0,12.0,25.0,0.0,0.0
2,Released,222.0,1.0,51.0,2.0,0.0,6.0,81.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,18,19,20,21,22
0,Completed,29.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
1,Executing,42.0,0.0,45.0,0.0,1.0,12.0,25.0,0.0,0.0
2,Released,222.0,1.0,51.0,2.0,0.0,6.0,81.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()